In [46]:
import numpy as np
import pandas as pd
import requests

In [11]:
#weather_dir = 'I:/Weather Data/'
weather_dir = '.'

In [12]:
daily_tmin_df = pd.read_fwf(f'{weather_dir}/dly-tmin-normal.txt', header=None)
daily_tmin_df.columns = ['STNID','MONTH'] + [i for i in range(1,32)]
daily_tmin_df = pd.melt(daily_tmin_df, id_vars = ['STNID','MONTH'], value_vars = [i for i in range(1,32)])
daily_tmin_df.columns = ['STNID','MONTH','DAY','VALUE']
daily_tmin_df.head()

,STNID,MONTH,DAY,VALUE
0,AQW00061705,1,1,772C
1,AQW00061705,2,1,774C
2,AQW00061705,3,1,776C
3,AQW00061705,4,1,774C
4,AQW00061705,5,1,773C


In [13]:
daily_tmin_df = daily_tmin_df[daily_tmin_df['VALUE'] != '-8888']
daily_tmin_df['FLAG'] = daily_tmin_df['VALUE'].map(lambda x: x[-1])
daily_tmin_df['VALUE'] = daily_tmin_df['VALUE'].map(lambda x: int(x[0:-1])/10)
daily_tmin_df

,STNID,MONTH,DAY,VALUE,FLAG
0,AQW00061705,1,1,77.2,C
1,AQW00061705,2,1,77.4,C
2,AQW00061705,3,1,77.6,C
3,AQW00061705,4,1,77.4,C
4,AQW00061705,5,1,77.3,C
5,AQW00061705,6,1,77.2,C
6,AQW00061705,7,1,76.5,C
7,AQW00061705,8,1,76.2,C
8,AQW00061705,9,1,76.5,C
9,AQW00061705,10,1,76.8,C


In [14]:
daily_tmax_df = pd.read_fwf(f'{weather_dir}/dly-tmax-normal.txt', header=None)
daily_tmax_df.columns = ['STNID','MONTH'] + [i for i in range(1,32)]
daily_tmax_df = pd.melt(daily_tmax_df, id_vars = ['STNID','MONTH'], value_vars = [i for i in range(1,32)])
daily_tmax_df.columns = ['STNID','MONTH','DAY','VALUE']

daily_tmax_df = daily_tmax_df[daily_tmax_df['VALUE'] != '-8888']
daily_tmax_df['FLAG'] = daily_tmax_df['VALUE'].map(lambda x: x[-1])
daily_tmax_df['VALUE'] = daily_tmax_df['VALUE'].map(lambda x: int(x[0:-1])/10)
daily_tmax_df

,STNID,MONTH,DAY,VALUE,FLAG
0,AQW00061705,1,1,87.5,C
1,AQW00061705,2,1,87.8,C
2,AQW00061705,3,1,88.2,C
3,AQW00061705,4,1,87.9,C
4,AQW00061705,5,1,86.9,C
5,AQW00061705,6,1,85.5,C
6,AQW00061705,7,1,84.6,C
7,AQW00061705,8,1,84.0,C
8,AQW00061705,9,1,84.9,C
9,AQW00061705,10,1,85.6,C


In [62]:
precip_df = pd.read_fwf(f'{weather_dir}/ann-prcp-normal.txt', header=None)
precip_df.columns = ['STNID','VALUE']
precip_df['FLAG'] = precip_df['VALUE'].map(lambda x: x[-1])
precip_df['annual_precipitation'] = precip_df['VALUE'].map(lambda x: int(x[0:-1])/100)

In [63]:
df = pd.DataFrame({
    'min_temp': daily_temp_df.groupby('STNID')['VALUE_MIN'].min(), 
    'max_temp':daily_temp_df.groupby('STNID')['VALUE_MAX'].max()
}).reset_index()
df = pd.merge(df, precip_df, on='STNID')
df = df[df['STNID'].str.startswith('US')]

In [64]:
all_stations_df = pd.read_fwf(f'{weather_dir}/allstations.txt', header=None)
all_stations_df.columns = ['ID', 'Latitude','Longitude','Elevation','State','Name','GSNFLAG','HCNFLAG','WMOID']
all_stations_df.head()

,ID,Latitude,Longitude,Elevation,State,Name,GSNFLAG,HCNFLAG,WMOID
0,AQC00914000,-14.3167,-170.7667,408.4,AS,AASUFOU,NaN,NaN,NaN
1,AQW00061705,-14.3306,-170.7136,3.7,AS,PAGO PAGO WSO AP,NaN,GSN,91765.0
2,CAW00064757,44.2325,-79.7811,246.0,ON,EGBERT 1 W,NaN,NaN,NaN
3,CQC00914080,15.2136,145.7497,252.1,MP,CAPITOL HILL 1,NaN,NaN,NaN
4,CQC00914801,14.1717,145.2428,179.2,MP,ROTA AP,NaN,NaN,91221.0


In [65]:
stations = pd.merge(df,all_stations_df,left_on='STNID',right_on='ID')
stations

,STNID,min_temp,max_temp,VALUE,FLAG,annual_precipitation,ID,Latitude,Longitude,Elevation,State,Name,GSNFLAG,HCNFLAG,WMOID
0,USC00010063,28.5,90.1,5824R,R,58.24,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN
1,USC00010160,31.4,90.9,5562S,S,55.62,USC00010160,32.9453,-85.9481,195.1,AL,ALEXANDER CITY,NaN,NaN,NaN
2,USC00010178,31.0,92.7,5617R,R,56.17,USC00010178,33.1272,-88.1550,59.4,AL,ALICEVILLE,NaN,NaN,NaN
3,USC00010252,31.9,91.8,5997R,R,59.97,USC00010252,31.3072,-86.5225,76.2,AL,ANDALUSIA 3 W,NaN,NaN,NaN
4,USC00010260,26.4,89.8,5793Q,Q,57.93,USC00010260,34.9092,-87.2747,208.8,AL,ANDERSON,NaN,NaN,NaN
5,USC00010369,29.4,89.9,5815R,R,58.15,USC00010369,33.2942,-85.7789,311.5,AL,ASHLAND 3 ENE,NaN,NaN,NaN
6,USC00010390,29.8,90.2,5689R,R,56.89,USC00010390,34.7753,-86.9508,210.0,AL,ATHENS,NaN,NaN,NaN
7,USC00010402,36.9,91.3,6269R,R,62.69,USC00010402,31.1819,-87.4389,91.4,AL,ATMORE,NaN,NaN,NaN
8,USC00010425,32.2,90.5,5357R,R,53.57,USC00010425,32.5992,-85.4653,166.1,AL,AUBURN NO.2,NaN,NaN,NaN
9,USC00010430,33.5,91.1,5255R,R,52.55,USC00010430,32.6000,-85.5000,198.7,AL,AUBURN AGRONOMY FARM,NaN,NaN,NaN


In [55]:
list(stations.columns)

['STNID',
 'min_temp',
 'max_temp',
 'ID',
 'Latitude',
 'Longitude',
 'Elevation',
 'State',
 'Name',
 'GSNFLAG',
 'HCNFLAG',
 'WMOID']

In [37]:
lat_longs = list(stations.apply(lambda row: {'Latitude':row['Latitude'],'Longitude':row['Longitude']}, axis='columns'))
lat_longs

for lat_long in lat_longs:
    url = f"https://geo.fcc.gov/api/census/block/find?latitude={lat_long['Latitude']}&longitude={lat_long['Longitude']}&showall=false&format=json"
    response = requests.get(url)
    if response.ok:
        lat_long['FIPS_COUNTY'] = response.json()['County']['FIPS']
    print(lat_long)
    
# for row in stations.iterrows():
#     #url = f"https://geo.fcc.gov/api/census/block/find?latitude={row['Latitude']}&longitude={row['Longitude']}&showall=false&format=json"
#     url = f"https://geo.fcc.gov/api/census/block/find?latitude={row['Latitude']}&longitude=&showall=false&format=json"
#     print(url)
#     #response = requests.get

{'Latitude': 34.2553, 'Longitude': -87.1814, 'FIPS_COUNTY': '01133'}
{'Latitude': 32.9453, 'Longitude': -85.9481, 'FIPS_COUNTY': '01123'}
{'Latitude': 33.1272, 'Longitude': -88.155, 'FIPS_COUNTY': '01107'}
{'Latitude': 31.3072, 'Longitude': -86.5225, 'FIPS_COUNTY': '01039'}
{'Latitude': 34.9092, 'Longitude': -87.2747, 'FIPS_COUNTY': '01077'}
{'Latitude': 33.2942, 'Longitude': -85.7789, 'FIPS_COUNTY': '01027'}
{'Latitude': 34.7753, 'Longitude': -86.9508, 'FIPS_COUNTY': '01083'}
{'Latitude': 31.1819, 'Longitude': -87.4389, 'FIPS_COUNTY': '01053'}
{'Latitude': 32.5992, 'Longitude': -85.4653, 'FIPS_COUNTY': '01081'}
{'Latitude': 32.6, 'Longitude': -85.5, 'FIPS_COUNTY': '01081'}
{'Latitude': 33.4528, 'Longitude': -87.3572, 'FIPS_COUNTY': '01125'}
{'Latitude': 30.8839, 'Longitude': -87.7853, 'FIPS_COUNTY': '01003'}
{'Latitude': 34.6908, 'Longitude': -86.8825, 'FIPS_COUNTY': '01083'}
{'Latitude': 33.3953, 'Longitude': -87.0078, 'FIPS_COUNTY': '01073'}
{'Latitude': 31.1414, 'Longitude': -87.04

{'Latitude': 33.2058, 'Longitude': -111.6819, 'FIPS_COUNTY': '04013'}
{'Latitude': 34.54, 'Longitude': -110.9153, 'FIPS_COUNTY': '04005'}
{'Latitude': 34.3494, 'Longitude': -111.6981, 'FIPS_COUNTY': '04025'}
{'Latitude': 34.7569, 'Longitude': -112.4567, 'FIPS_COUNTY': '04025'}
{'Latitude': 32.0061, 'Longitude': -109.3572, 'FIPS_COUNTY': '04003'}
{'Latitude': 33.0561, 'Longitude': -109.3072, 'FIPS_COUNTY': '04011'}
{'Latitude': 36.9936, 'Longitude': -112.9719, 'FIPS_COUNTY': '04015'}
{'Latitude': 34.305, 'Longitude': -112.165, 'FIPS_COUNTY': '04025'}
{'Latitude': 31.3456, 'Longitude': -110.2542, 'FIPS_COUNTY': '04003'}
{'Latitude': 34.2097, 'Longitude': -112.3453, 'FIPS_COUNTY': '04025'}
{'Latitude': 32.9711, 'Longitude': -113.4981, 'FIPS_COUNTY': '04027'}
{'Latitude': 35.2947, 'Longitude': -113.3664, 'FIPS_COUNTY': '04015'}
{'Latitude': 31.345, 'Longitude': -109.5394, 'FIPS_COUNTY': '04003'}
{'Latitude': 32.7481, 'Longitude': -109.1214, 'FIPS_COUNTY': '04011'}
{'Latitude': 33.4192, 'Lo

{'Latitude': 32.2553, 'Longitude': -109.8369, 'FIPS_COUNTY': '04003'}
{'Latitude': 35.2406, 'Longitude': -112.1903, 'FIPS_COUNTY': '04005'}
{'Latitude': 35.8686, 'Longitude': -114.6611, 'FIPS_COUNTY': '04015'}
{'Latitude': 35.6169, 'Longitude': -109.1244, 'FIPS_COUNTY': '04001'}
{'Latitude': 33.7478, 'Longitude': -112.5983, 'FIPS_COUNTY': '04013'}
{'Latitude': 35.5247, 'Longitude': -111.3703, 'FIPS_COUNTY': '04005'}
{'Latitude': 33.595, 'Longitude': -112.3014, 'FIPS_COUNTY': '04013'}
{'Latitude': 34.8775, 'Longitude': -114.1344, 'FIPS_COUNTY': '04015'}
{'Latitude': 32.6114, 'Longitude': -114.635, 'FIPS_COUNTY': '04027'}
{'Latitude': 32.7278, 'Longitude': -114.6217, 'FIPS_COUNTY': '04027'}
{'Latitude': 35.9289, 'Longitude': -91.0583, 'FIPS_COUNTY': '05075'}
{'Latitude': 34.7961, 'Longitude': -92.8417, 'FIPS_COUNTY': '05125'}
{'Latitude': 34.1433, 'Longitude': -93.0589, 'FIPS_COUNTY': '05019'}
{'Latitude': 34.025, 'Longitude': -91.3444, 'FIPS_COUNTY': '05001'}
{'Latitude': 35.8306, 'Long

{'Latitude': 37.0856, 'Longitude': -122.0797, 'FIPS_COUNTY': '06087'}
{'Latitude': 37.8744, 'Longitude': -122.2606, 'FIPS_COUNTY': '06001'}
{'Latitude': 40.7406, 'Longitude': -123.2078, 'FIPS_COUNTY': '06105'}
{'Latitude': 34.2442, 'Longitude': -116.9039, 'FIPS_COUNTY': '06071'}
{'Latitude': 37.2064, 'Longitude': -119.2419, 'FIPS_COUNTY': '06019'}
{'Latitude': 36.2472, 'Longitude': -121.7803, 'FIPS_COUNTY': '06053'}
{'Latitude': 33.6131, 'Longitude': -114.5972, 'FIPS_COUNTY': '06065'}
{'Latitude': 39.3886, 'Longitude': -120.0936, 'FIPS_COUNTY': '06057'}
{'Latitude': 38.2119, 'Longitude': -119.0142, 'FIPS_COUNTY': '06051'}
{'Latitude': 33.2314, 'Longitude': -116.4144, 'FIPS_COUNTY': '06073'}
{'Latitude': 39.4539, 'Longitude': -120.6556, 'FIPS_COUNTY': '06057'}
{'Latitude': 32.9544, 'Longitude': -115.5581, 'FIPS_COUNTY': '06025'}
{'Latitude': 38.2575, 'Longitude': -119.2286, 'FIPS_COUNTY': '06051'}
{'Latitude': 40.8803, 'Longitude': -121.6547, 'FIPS_COUNTY': '06089'}
{'Latitude': 35.4047

{'Latitude': 36.3817, 'Longitude': -119.0264, 'FIPS_COUNTY': '06107'}
{'Latitude': 36.2033, 'Longitude': -119.0545, 'FIPS_COUNTY': '06107'}
{'Latitude': 37.6922, 'Longitude': -121.7692, 'FIPS_COUNTY': '06001'}
{'Latitude': 36.6044, 'Longitude': -118.7325, 'FIPS_COUNTY': '06107'}
{'Latitude': 38.1061, 'Longitude': -121.2878, 'FIPS_COUNTY': '06077'}
{'Latitude': 34.6539, 'Longitude': -120.4514, 'FIPS_COUNTY': '06083'}
{'Latitude': 37.0564, 'Longitude': -120.8667, 'FIPS_COUNTY': '06047'}
{'Latitude': 36.9897, 'Longitude': -120.9297, 'FIPS_COUNTY': '06047'}
{'Latitude': 37.2319, 'Longitude': -121.9592, 'FIPS_COUNTY': '06085'}
{'Latitude': 36.9539, 'Longitude': -120.0378, 'FIPS_COUNTY': '06039'}
{'Latitude': 37.6478, 'Longitude': -118.9617, 'FIPS_COUNTY': '06051'}
{'Latitude': 40.5411, 'Longitude': -121.5767, 'FIPS_COUNTY': '06089'}
{'Latitude': 38.4917, 'Longitude': -122.1242, 'FIPS_COUNTY': '06055'}
{'Latitude': 38.0131, 'Longitude': -122.1142, 'FIPS_COUNTY': '06013'}
{'Latitude': 39.1458

...

{'Latitude': 44.1278, 'Longitude': -123.2206, 'FIPS_COUNTY': '41039'}
{'Latitude': 47.9078, 'Longitude': -122.2803, 'FIPS_COUNTY': '53061'}
{'Latitude': 42.3811, 'Longitude': -122.8722, 'FIPS_COUNTY': '41029'}
{'Latitude': 46.9733, 'Longitude': -122.9033, 'FIPS_COUNTY': '53067'}
{'Latitude': 45.5908, 'Longitude': -122.6003, 'FIPS_COUNTY': '41051'}
{'Latitude': 44.2558, 'Longitude': -121.1392, 'FIPS_COUNTY': '41017'}
{'Latitude': 43.2389, 'Longitude': -123.3547, 'FIPS_COUNTY': '41019'}
{'Latitude': 44.905, 'Longitude': -123.0011, 'FIPS_COUNTY': '41047'}
{'Latitude': 47.4444, 'Longitude': -122.3139, 'FIPS_COUNTY': '53033'}
{'Latitude': 47.5303, 'Longitude': -122.3008, 'FIPS_COUNTY': '53033'}
{'Latitude': 42.6003, 'Longitude': -123.3642, 'FIPS_COUNTY': '41033'}
{'Latitude': 47.2767, 'Longitude': -121.3372, 'FIPS_COUNTY': '53037'}
{'Latitude': 46.4833, 'Longitude': -122.8167, 'FIPS_COUNTY': '53041'}
{'Latitude': 45.5511, 'Longitude': -122.4089, 'FIPS_COUNTY': '41051'}
{'Latitude': 46.5683,

{'Latitude': 35.1967, 'Longitude': -81.1558, 'FIPS_COUNTY': '37071'}
{'Latitude': 34.9869, 'Longitude': -81.0575, 'FIPS_COUNTY': '45091'}
{'Latitude': 35.0169, 'Longitude': -80.6206, 'FIPS_COUNTY': '37179'}
{'Latitude': 34.1231, 'Longitude': -84.8486, 'FIPS_COUNTY': '13015'}
{'Latitude': 34.2486, 'Longitude': -82.1592, 'FIPS_COUNTY': '45047'}
{'Latitude': 35.4944, 'Longitude': -82.6142, 'FIPS_COUNTY': '37021'}
{'Latitude': 35.4186, 'Longitude': -82.5567, 'FIPS_COUNTY': '37089'}
{'Latitude': 38.9597, 'Longitude': -94.3714, 'FIPS_COUNTY': '29095'}
{'Latitude': 36.8781, 'Longitude': -91.9025, 'FIPS_COUNTY': '29091'}
{'Latitude': 30.3567, 'Longitude': -95.4139, 'FIPS_COUNTY': '48339'}
{'Latitude': 30.7439, 'Longitude': -95.5861, 'FIPS_COUNTY': '48471'}
{'Latitude': 38.9286, 'Longitude': -90.4281, 'FIPS_COUNTY': '29183'}
{'Latitude': 32.5428, 'Longitude': -93.745, 'FIPS_COUNTY': '22015'}
{'Latitude': 32.6636, 'Longitude': -97.0939, 'FIPS_COUNTY': '48439'}
{'Latitude': 36.0394, 'Longitude': 

{'Latitude': 26.25, 'Longitude': -80.1083, 'FIPS_COUNTY': '12011'}
{'Latitude': 27.7628, 'Longitude': -82.6261, 'FIPS_COUNTY': '12103'}
{'Latitude': 25.9989, 'Longitude': -80.2411, 'FIPS_COUNTY': '12011'}
{'Latitude': 25.8064, 'Longitude': -80.1336, 'FIPS_COUNTY': '12086'}
{'Latitude': 28.6158, 'Longitude': -80.6928, 'FIPS_COUNTY': '12009'}
{'Latitude': 25.8997, 'Longitude': -81.3183, 'FIPS_COUNTY': '12021'}
{'Latitude': 27.1525, 'Longitude': -81.3689, 'FIPS_COUNTY': '12055'}
{'Latitude': 37.1431, 'Longitude': -107.7597, 'FIPS_COUNTY': '08067'}
{'Latitude': 39.2292, 'Longitude': -106.3169, 'FIPS_COUNTY': '08065'}
{'Latitude': 39.1894, 'Longitude': -103.7158, 'FIPS_COUNTY': '08039'}
{'Latitude': 38.505, 'Longitude': -107.8975, 'FIPS_COUNTY': '08085'}
{'Latitude': 31.4583, 'Longitude': -109.6061, 'FIPS_COUNTY': '04003'}
{'Latitude': 34.5183, 'Longitude': -109.3792, 'FIPS_COUNTY': '04001'}
{'Latitude': 32.3336, 'Longitude': -104.2581, 'FIPS_COUNTY': '35015'}
{'Latitude': 38.81, 'Longitude

{'Latitude': 32.7817, 'Longitude': -98.0603, 'FIPS_COUNTY': '48367'}
{'Latitude': 34.9894, 'Longitude': -99.0525, 'FIPS_COUNTY': '40075'}
{'Latitude': 31.2361, 'Longitude': -94.7544, 'FIPS_COUNTY': '48005'}
{'Latitude': 34.175, 'Longitude': -91.9347, 'FIPS_COUNTY': '05069'}
{'Latitude': 39.9369, 'Longitude': -91.1919, 'FIPS_COUNTY': '17001'}
{'Latitude': 39.3756, 'Longitude': -99.8297, 'FIPS_COUNTY': '20065'}
{'Latitude': 33.2208, 'Longitude': -92.8142, 'FIPS_COUNTY': '05139'}
{'Latitude': 36.0097, 'Longitude': -94.1694, 'FIPS_COUNTY': '05143'}
{'Latitude': 38.8761, 'Longitude': -98.8092, 'FIPS_COUNTY': '20167'}
{'Latitude': 48.2064, 'Longitude': -106.6242, 'FIPS_COUNTY': '30105'}
{'Latitude': 48.4089, 'Longitude': -106.5144, 'FIPS_COUNTY': '30105'}
{'Latitude': 48.5428, 'Longitude': -109.7633, 'FIPS_COUNTY': '30041'}
{'Latitude': 48.1739, 'Longitude': -103.6367, 'FIPS_COUNTY': '38105'}
{'Latitude': 48.0944, 'Longitude': -105.5744, 'FIPS_COUNTY': '30085'}
{'Latitude': 44.3394, 'Longitu

{'Latitude': 41.3536, 'Longitude': -96.0233, 'FIPS_COUNTY': '31055'}
{'Latitude': 46.5486, 'Longitude': -90.9189, 'FIPS_COUNTY': '55003'}
{'Latitude': 47.3864, 'Longitude': -92.8389, 'FIPS_COUNTY': '27137'}
{'Latitude': 46.4047, 'Longitude': -94.1308, 'FIPS_COUNTY': '27035'}
{'Latitude': 43.7667, 'Longitude': -99.3183, 'FIPS_COUNTY': '46015'}
{'Latitude': 41.4333, 'Longitude': -99.6333, 'FIPS_COUNTY': '31041'}
{'Latitude': 40.6044, 'Longitude': -98.4272, 'FIPS_COUNTY': '31001'}
{'Latitude': 43.7667, 'Longitude': -98.0333, 'FIPS_COUNTY': '46035'}
{'Latitude': 40.0803, 'Longitude': -95.5919, 'FIPS_COUNTY': '31147'}
{'Latitude': 41.6236, 'Longitude': -98.9517, 'FIPS_COUNTY': '31175'}
{'Latitude': 45.0619, 'Longitude': -93.3508, 'FIPS_COUNTY': '27053'}
{'Latitude': 48.7167, 'Longitude': -94.6, 'FIPS_COUNTY': '27077'}
{'Latitude': 44.8322, 'Longitude': -93.4706, 'FIPS_COUNTY': '27053'}
{'Latitude': 46.9006, 'Longitude': -95.0678, 'FIPS_COUNTY': '27057'}
{'Latitude': 43.4075, 'Longitude': -9

In [50]:
avg_weekly_wage = pd.read_csv('avg_weekly_wage.csv', dtype={'FIPS_COUNTY':str})
avg_weekly_wage.head()

,FIPS_COUNTY,COUNTY_NAME,AVG_WEEKLY_WAGE
0,01001,"Autauga County, Alabama",802
1,01003,"Baldwin County, Alabama",717
2,01005,"Barbour County, Alabama",727
3,01007,"Bibb County, Alabama",785
4,01009,"Blount County, Alabama",683


In [66]:
data_df = pd.merge(stations, pd.DataFrame(lat_longs), on=['Latitude','Longitude'])
data_df = pd.merge(data_df, avg_weekly_wage, on='FIPS_COUNTY')
data_df

,STNID,min_temp,max_temp,VALUE,FLAG,annual_precipitation,ID,Latitude,Longitude,Elevation,State,Name,GSNFLAG,HCNFLAG,WMOID,FIPS_COUNTY,COUNTY_NAME,AVG_WEEKLY_WAGE
0,USC00010063,28.5,90.1,5824R,R,58.24,USC00010063,34.2553,-87.1814,249.3,AL,ADDISON,NaN,NaN,NaN,01133,"Winston County, Alabama",721
1,USC00010160,31.4,90.9,5562S,S,55.62,USC00010160,32.9453,-85.9481,195.1,AL,ALEXANDER CITY,NaN,NaN,NaN,01123,"Tallapoosa County, Alabama",645
2,USC00011324,25.8,91.7,5366Q,Q,53.66,USC00011324,32.8236,-85.6561,207.3,AL,CAMP HILL 2NW,NaN,NaN,NaN,01123,"Tallapoosa County, Alabama",645
3,USC00010178,31.0,92.7,5617R,R,56.17,USC00010178,33.1272,-88.1550,59.4,AL,ALICEVILLE,NaN,NaN,NaN,01107,"Pickens County, Alabama",735
4,USC00010252,31.9,91.8,5997R,R,59.97,USC00010252,31.3072,-86.5225,76.2,AL,ANDALUSIA 3 W,NaN,NaN,NaN,01039,"Covington County, Alabama",693
5,USC00010260,26.4,89.8,5793Q,Q,57.93,USC00010260,34.9092,-87.2747,208.8,AL,ANDERSON,NaN,NaN,NaN,01077,"Lauderdale County, Alabama",674
6,USC00010369,29.4,89.9,5815R,R,58.15,USC00010369,33.2942,-85.7789,311.5,AL,ASHLAND 3 ENE,NaN,NaN,NaN,01027,"Clay County, Alabama",661
7,USC00010390,29.8,90.2,5689R,R,56.89,USC00010390,34.7753,-86.9508,210.0,AL,ATHENS,NaN,NaN,NaN,01083,"Limestone County, Alabama",917
8,USC00010655,27.3,91.2,5321R,R,53.21,USC00010655,34.6908,-86.8825,182.9,AL,BELLE MINA 2 N,NaN,NaN,NaN,01083,"Limestone County, Alabama",917
9,USW00053852,30.7,90.7,5338Q,Q,53.38,USW00053852,34.6525,-86.9453,180.4,AL,DECATUR PRYOR FLD,NaN,NaN,NaN,01083,"Limestone County, Alabama",917


In [68]:
data_df[['STNID','min_temp','max_temp','annual_precipitation','Latitude','Longitude','Name','FIPS_COUNTY','COUNTY_NAME','AVG_WEEKLY_WAGE']].to_csv('data.csv')

In [69]:
data_df['annual_precipitation'].describe()

count    6923.000000
mean       33.988753
std        18.226571
min         2.360000
25%        17.880000
50%        34.810000
75%        46.455000
max       240.020000
Name: annual_precipitation, dtype: float64